In [ ]:
from dataclasses import dataclass
import math


def norm_cdf(x):
    """Cumulative distribution function for standard normal distribution."""
    return (1 + (1 / math.sqrt(2 * math.pi)) * math.exp(-x ** 2 / 2)) / 2


@dataclass
class Equity:
    spot: float            # Current spot price of the stock
    dividend_yield: float  # Dividend yield as a decimal (e.g., 0.02 for 2%)
    volatility: float      # Annualized volatility as a decimal (e.g., 0.2 for 20%)


@dataclass
class EquityOption:
    strike: float          # Strike price of the option
    time_to_maturity: float  # Time to maturity in years
    put_call: str          # Option type: 'call' or 'put'


class Calculator:
    @staticmethod
    def add(x, y):
        return x + y

    @staticmethod
    def subtract(x, y):
        return x - y

    @staticmethod
    def multiply(x, y):
        return x * y

    @staticmethod
    def divide(x, y):
        if y == 0:
            raise ValueError("Cannot divide by zero")
        return x / y

    @staticmethod
    def square(x):
        return x * x

    @staticmethod
    def root(x):
        if x < 0:
            raise ValueError("x cannot be negative")
        return math.sqrt(x)

    @staticmethod
    def bsm_pricer(equity: Equity, option: EquityOption, rate: float) -> float:
        """Black-Scholes-Merton (BSM) pricer for European options."""
        d1 = (
            math.log(equity.spot / option.strike)
            + (rate - equity.dividend_yield + 0.5 * Calculator.square(equity.volatility)) * option.time_to_maturity
        ) / (equity.volatility * Calculator.root(option.time_to_maturity))

        d2 = d1 - equity.volatility * Calculator.root(option.time_to_maturity)

        if option.put_call == 'call':
            price = (
                equity.spot * math.exp(-equity.dividend_yield * option.time_to_maturity) * norm_cdf(d1)
                - option.strike * math.exp(-rate * option.time_to_maturity) * norm_cdf(d2)
            )
        elif option.put_call == 'put':
            price = (
                option.strike * math.exp(-rate * option.time_to_maturity) * norm_cdf(-d2)
                - equity.spot * math.exp(-equity.dividend_yield * option.time_to_maturity) * norm_cdf(-d1)
            )
        else:
            raise ValueError("Invalid put_call entry. Use 'call' or 'put'.")

        return price


# Example Usage:
if __name__ == "__main__":
    equity = Equity(spot=150.0, dividend_yield=0.01, volatility=0.2)
    option = EquityOption(strike=160.0, time_to_maturity=1.0, put_call='call')
    rate = 0.03

    calculator = Calculator()
    price = calculator.bsm_pricer(equity, option, rate)
    print(f"The price of the {option.put_call} option is: {price:.2f}")
